In [1]:
import torchvision
import torch
import torch.nn as nn
import numpy as np
from torch import optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from skimage import io
import time
import ast 
from PIL import *
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
# subset / old csv
# df = pd.read_csv('temp.csv')

# area larger than 1000
# df = pd.read_csv('area_larger_than_1000.csv')

# full dataset 
df = pd.read_csv('large_nodules_png_bounding_box.csv')

# subset of dataset (25%)
# _ , X_random, = train_test_split(df, test_size=0.25, random_state=0)
# X_random.to_csv('subset.csv',index=False)
# df = X_random

df

,path,imageName,SOPInstanceUID,boxes,areas
0,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000059.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.319084926500...,"[[165.0, 140.0, 199.0, 161.0]]",714.0
1,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000008.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.749728215582...,"[[167.0, 134.0, 201.0, 160.0]]",884.0
2,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000109.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.327906493223...,"[[167.0, 134.0, 202.0, 161.0]]",945.0
3,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000063.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.230680133701...,"[[168.0, 135.0, 201.0, 160.0]]",825.0
4,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000005.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.212532866259...,"[[180.0, 133.0, 200.0, 154.0]]",420.0
...,...,...,...,...,...
9395,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000017.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.323491922755...,"[[373.0, 352.0, 384.0, 364.0]]",132.0
9396,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000082.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.276843939440...,"[[346.0, 350.0, 361.0, 361.0]]",165.0
9397,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000133.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.207293484767...,"[[345.0, 351.0, 358.0, 361.0]]",130.0
9398,/scratch/ebc308/tcia/data_png/LIDC-IDRI/LIDC-I...,000051.png,1.3.6.1.4.1.14519.5.2.1.6279.6001.518873247105...,"[[374.0, 354.0, 384.0, 362.0]]",80.0


In [4]:
# split dataset into 60, 20, 20 
train_data, test_data = train_test_split(df, test_size=0.40, random_state=2020)
valid_data, test_data = train_test_split(test_data, test_size=0.50, random_state=2020)
train_data.index = np.arange(len(train_data))
valid_data.index = np.arange(len(valid_data))
test_data.index = np.arange(len(test_data))

In [5]:
train_data.to_csv("large_nodules_train.csv", index=False)
valid_data.to_csv("large_nodules_valid.csv", index=False)
test_data.to_csv("large_nodules_test.csv", index=False)

In [6]:
from torchvision import transforms
# torchvision models are trained on input images normalized to [0 1] range .ToPILImage() function achives this
# additional normalization is required see: http://pytorch.org/docs/master/torchvision/models.html

train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(896),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

validation_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.CenterCrop(896),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

class Faster_RCNN_Dataloader(Dataset):
    """Chest X-ray dataset from https://nihcc.app.box.com/v/ChestXray-NIHCC."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file filename information.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        
        # get image path
        img_name = (self.data_frame.iloc[idx, 0])
        
        # open image 
        image = io.imread(img_name)
        
        # 3 channels
        if len(image.shape) > 2 and image.shape[2] == 4:
            image = image[:,:,0]       
        image = np.repeat(image[None,...],3,axis=0)
        
        # transform image? 
        if self.transform:
            image = self.transform(np.uint8(image))

        d = {}
        d['boxes'] = torch.FloatTensor(ast.literal_eval(self.data_frame.iloc[idx, 3]))
        d['labels'] = torch.ones([1], dtype=torch.int64)

        return image, d

# change dataloader output to lists.
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    return list(xx), list(yy)

In [7]:
train_df_path = 'large_nodules_train.csv'
validate_df_path = 'large_nodules_valid.csv'
test_df_path = 'large_nodules_test.csv'

transformed_dataset = {'train': Faster_RCNN_Dataloader(train_df_path, transform=train_transform),
                       'validate':Faster_RCNN_Dataloader(validate_df_path, transform=validation_transform),
                       'test':Faster_RCNN_Dataloader(test_df_path)}
bs = 8

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs,
                        shuffle=True, collate_fn = pad_collate, num_workers=0) for x in ['train', 'validate','test']}
data_sizes ={x: len(transformed_dataset[x]) for x in ['train', 'validate','test']}


# one dataloader / phase, may be too memory intensive...

# train_dataset = {'train': Faster_RCNN_Dataloader(train_df_path, transform=train_transform)}
# validate_dataset = {'validate': Faster_RCNN_Dataloader(validate_df_path, transform=validation_transform)}
# test_dataset = {'test': Faster_RCNN_Dataloader(test_df_path)}

# bs = 16 #if bigger, running out of memory in p100

# train_dataloader = {x : DataLoader(train_dataset[x], batch_size=bs,
#                         shuffle=True,collate_fn = pad_collate, num_workers=0) for x in ['train']}
# validate_dataloader = {x : DataLoader(validate_dataset[x], batch_size=bs,
#                         shuffle=True,collate_fn = pad_collate, num_workers=0) for x in ['validate']}
# test_dataloader = {x : DataLoader(test_dataset[x], batch_size=bs,
#                         shuffle=True,collate_fn = pad_collate, num_workers=0) for x in ['test']}
# data_sizes ={len(dataset[x]) for x in ['train']}

In [8]:
# check if dataloader loaded correctly
# images1, targets1 = next(iter(train_dataloader['train']))
# images2, targets2 = next(iter(validate_dataloader['validate']))
# images3, targets3 = next(iter(test_dataloader['test']))

# to check dim of list dataloader
# len(images) = batch
# len(images[0]) = channel
# len(images[0]) = height
# len(images[0]) = width
images4, targets4 = next(iter(dataloader['train']))
len(images4[0])

3

In [9]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (nodule) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [12]:
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# # load a pre-trained model for classification and return
# # only the features
# backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone = torchvision.models.vgg16(pretrained=True).features

# # FasterRCNN needs to know the number of
# # output channels in a backbone. For mobilenet_v2, it's 1280
# # so we need to add it here
# backbone.out_channels = 1280
backbone.out_channels = 512

# # let's make the RPN generate 5 x 3 anchors per spatial
# # location, with 5 different sizes and 3 different aspect
# # ratios. We have a Tuple[Tuple[int]] because each feature
# # map could potentially have different sizes and
# # aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# # let's define what are the feature maps that we will
# # use to perform the region of interest cropping, as well as
# # the size of the crop after rescaling.
# # if your backbone returns a Tensor, featmap_names is expected to
# # be [0]. More generally, the backbone should return an
# # OrderedDict[Tensor], and in featmap_names you can choose which
# # feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=["0"],
                                                output_size=7,
                                                sampling_ratio=2)

model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler).to(device)

# # tried to run not pretrained.
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=2).to(device)

# for multiple nodes
torch.nn.DataParallel(model)


optimizer = torch.optim.SGD(model.parameters(), lr=1e-5,
                                momentum=0.9, weight_decay=1e-5)

# optimizer = torch.optim.SGD(model.parameters(), lr=0.00001, momentum=0.9, weight_decay=0.0005)

lambda_func = lambda epoch: 0.5 ** epoch
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_func)

# unused 
cel = nn.MSELoss() 

In [13]:
# move dict/lists to gpu 
def move_to(obj, device):
    if torch.is_tensor(obj):
        return obj.to(device)
    elif isinstance(obj, dict):
        res = {}
        for k, v in obj.items():
            res[k] = move_to(v, device)
        return res
    elif isinstance(obj, list):
        res = []
        for v in obj:
            res.append(move_to(v, device))
        return res
    else:
        raise TypeError("Invalid type for move_to")

In [15]:
# def calc_iou( gt_bbox, pred_bbox):
#     '''
#     This function takes the predicted bounding box and ground truth bounding box and 
#     return the IoU ratio
#     '''
    
#     # xmin ymax xmax ymin
#     x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt= gt_bbox
    
#     # x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p= pred_bbox
#     x1, y1, x2, y2 = pred_bbox
#     x_topleft_p = min(x1, x2)
#     y_topleft_p = max(y1, y2)
#     x_bottomright_p = max(x1, x2)
#     y_bottomright_p = min(y1, y2)
    
# #     if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt> y_bottomright_gt):
# # #         raise AssertionError("Ground Truth Bounding Box is not correct")
# #         return "Ground Truth Bounding Box is not correct"
# #     if (x_topleft_p > x_bottomright_p) or (y_topleft_p> y_bottomright_p):
# # #         raise AssertionError("Predicted Bounding Box is not correct",x_topleft_p, x_bottomright_p,y_topleft_p,y_bottomright_gt)
# #         return "Predicted Bounding Box is not correct"
         
#     #if the GT bbox and predcited BBox do not overlap then iou=0
#     if(x_bottomright_gt< x_topleft_p):
#         # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox
        
#         return 0.0
#     if(y_bottomright_gt> y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox
        
#         return 0.0
#     if(x_topleft_gt> x_bottomright_p): # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox
        
#         return 0.0
#     if(y_topleft_gt< y_bottomright_p): # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox
        
#         return 0.0
    
#     # original code online calculation
# #     GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_bottomright_gt -y_topleft_gt + 1)
# #     Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_bottomright_p -y_topleft_p + 1)
    
#     # xmax - xmin , ymax - ymin
#     GT_bbox_area = (x_bottomright_gt -  x_topleft_gt + 1) * (  y_topleft_gt - y_bottomright_gt + 1)
#     Pred_bbox_area =(x_bottomright_p - x_topleft_p + 1 ) * ( y_topleft_p - y_bottomright_p + 1)
    
    
#     x_top_left =np.max([x_topleft_gt, x_topleft_p])
#     y_top_left = np.max([y_topleft_gt, y_topleft_p])
#     x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
#     y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])
    
#     intersection_area = (x_bottom_right- x_top_left + 1) * (y_bottom_right-y_top_left  + 1)
    
#     union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)
   
#     return intersection_area/union_area

In [16]:
def train_model(model, dataloader,optimizer, scheduler, loss_fn, num_epochs = 10, verbose = True):
    acc_dict = {'train':[]}
    loss_dict = {'train':[], 'validate':[]}
    best_acc = 0
    phases = ['train','validate']
    since = time.time()
    epochLossTrain = list()
#     epochAccTrain = list()
    epochLossValidate = list()
    epochAccValidate = list()
    for i in range(num_epochs):
        print('Epoch: {}/{}'.format(i, num_epochs-1))
        print('-'*10)
        losslist = []
        for p in phases:
            running_correct = 0
            running_loss = 0
            running_total = 0
            if p == 'train':
                model.train()
            else:
                model.eval()   
            num = 0 # calculate which batch --> enumerate not working
            for image, target in dataloader[p]:
                optimizer.zero_grad()
                # move data to gpu
                image = move_to(image,device)
                target = move_to(target,device)
                if (num % 25 == 0):
                    print('Train set | epoch: {:3d} | {:6d}/{:6d} batches'.format(i, num + 1, len(dataloader[p])))
                num = num + 1
                
                if p == 'train':
                    # loss = sum of all 4 losses returned
                    output = model((image), target)
                    loss = sum(loss for loss in output.values())
                else:
                    # helps with memory 
                    with torch.no_grad():
                        model.eval()  
                        output = model((image))
                        pred_score = [] # confidence score
                        pred_boxes = [] # bounding box
                        pred_class = [] # predicted class
                        maxIOUs = []
                        # loop through output (len(output) = batch size )
                        for j, out in enumerate(output):
                            # documentation: ['boxes'] = [x1, y1, x2, y2]
                            pred_boxes = [[i[0], i[1], i[2], i[3]] for i in list(out['boxes'].cpu().detach().numpy())]
                            pred_score = list(out['scores'].cpu().detach().numpy())
                            pred_class = [classes[i] for i in list(out['labels'].cpu().numpy())]
                            
                            # get correct label
                            # documentation: # 0 xmin, 1 ymin, 2 xmax, 3 ymax
                            t = target[j]['boxes'].cpu().detach().numpy()[0]
                            
                            # want to input: x min , y max, x max, y min
                            target_box =  [t[0],t[3],t[2],t[1]]
                            
                            # get index of max score - in theory want this one.
                            index = pred_score.index(max(pred_score))
                            
                            # right now, get iou of every predicted box
                            # and keep max iou 
                            iou = []
                            for ind in pred_boxes:
                                iou.append(calc_iou(target_box, ind))
                                maxIOUs.append(max(iou))
                        # sum max iou for each validate run to print out to see if above 0.0
                        epoch_acc = (sum(maxIOUs))
                # printing average loss / epoch
                num_imgs = len(image)
                running_loss += loss*num_imgs
                running_total += num_imgs
                
                if p == 'train':
                    loss.backward()
                    optimizer.step()
                    
            epoch_loss = float(running_loss/running_total)
            
            # keep epoch loss / accuracy (for validation)
            if p == 'train':
                epochLossTrain.append(epoch_loss)
#                 epochAccTrain.append(epoch_acc)
            if p == 'validate':
                epochLossValidate.append(epoch_loss)
                epochAccValidate.append(epoch_acc)
            if verbose or (i%10 == 0):
                if p == 'train':
                    print('Phase:{}, epoch loss: {:.4f}'.format(p, epoch_loss))
                if p == 'validate':
                    print('Phase:{}, epoch loss: {:.4f} | epoch accuracy: {:.4f}'.format(p, epoch_loss, epoch_acc))
            # old code
#             acc_dict[p].append(epoch_acc)
#             loss_dict[p].append(epoch_loss)      
#             if p == 'validate':
#                 if epoch_acc > best_acc:
#                     best_acc = epoch_acc
#                     best_model_wts = model.state_dict()
#             else:
            # schedule supposed to be under "else:" - not sure what it does...
            if scheduler:
                scheduler.step()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
#     print('Best val acc: {:4f}'.format(best_acc))
    
#     model.load_state_dict(best_model_wts)
#     return model, acc_dict, loss_dict, epochLossTrain, epochLossValidate, epochAccTrain, epochAccValidate
    return model, acc_dict, loss_dict, epochLossTrain ,epochLossValidate, epochAccValidate


In [ ]:
model, acc_dict, loss_dict, epochLossTrain,epochLossValidate,epochAccValidate = train_model(model, dataloader, optimizer, 
                                                                        scheduler, 
                                                                        cel, 
                                                                        num_epochs=100)


Epoch: 0/99
----------
Train set | epoch:   0 |      1/   705 batches
Train set | epoch:   0 |     26/   705 batches
Train set | epoch:   0 |     51/   705 batches
Train set | epoch:   0 |     76/   705 batches
Train set | epoch:   0 |    101/   705 batches
Train set | epoch:   0 |    126/   705 batches
Train set | epoch:   0 |    151/   705 batches
Train set | epoch:   0 |    176/   705 batches
Train set | epoch:   0 |    201/   705 batches
Train set | epoch:   0 |    226/   705 batches
Train set | epoch:   0 |    251/   705 batches
Train set | epoch:   0 |    276/   705 batches
Train set | epoch:   0 |    301/   705 batches
Train set | epoch:   0 |    326/   705 batches
Train set | epoch:   0 |    351/   705 batches
Train set | epoch:   0 |    376/   705 batches
Train set | epoch:   0 |    401/   705 batches
Train set | epoch:   0 |    426/   705 batches
Train set | epoch:   0 |    451/   705 batches
Train set | epoch:   0 |    476/   705 batches
Train set | epoch:   0 |    501/   70

Train set | epoch:   4 |    226/   705 batches
Train set | epoch:   4 |    251/   705 batches
Train set | epoch:   4 |    276/   705 batches
Train set | epoch:   4 |    301/   705 batches
Train set | epoch:   4 |    326/   705 batches
Train set | epoch:   4 |    351/   705 batches
Train set | epoch:   4 |    376/   705 batches
Train set | epoch:   4 |    401/   705 batches
Train set | epoch:   4 |    426/   705 batches
Train set | epoch:   4 |    451/   705 batches
Train set | epoch:   4 |    476/   705 batches
Train set | epoch:   4 |    501/   705 batches
Train set | epoch:   4 |    526/   705 batches
Train set | epoch:   4 |    551/   705 batches
Train set | epoch:   4 |    576/   705 batches
Train set | epoch:   4 |    601/   705 batches
Train set | epoch:   4 |    626/   705 batches
Train set | epoch:   4 |    651/   705 batches
Train set | epoch:   4 |    676/   705 batches
Train set | epoch:   4 |    701/   705 batches
Phase:train, epoch loss: 0.0900
Train set | epoch:   4 |    

Train set | epoch:   8 |    451/   705 batches
Train set | epoch:   8 |    476/   705 batches
Train set | epoch:   8 |    501/   705 batches
Train set | epoch:   8 |    526/   705 batches
Train set | epoch:   8 |    551/   705 batches
Train set | epoch:   8 |    576/   705 batches
Train set | epoch:   8 |    601/   705 batches
Train set | epoch:   8 |    626/   705 batches
Train set | epoch:   8 |    651/   705 batches
Train set | epoch:   8 |    676/   705 batches
Train set | epoch:   8 |    701/   705 batches
Phase:train, epoch loss: 0.0899
Train set | epoch:   8 |      1/   235 batches
Train set | epoch:   8 |     26/   235 batches
Train set | epoch:   8 |     51/   235 batches
Train set | epoch:   8 |     76/   235 batches
Train set | epoch:   8 |    101/   235 batches
Train set | epoch:   8 |    126/   235 batches
Train set | epoch:   8 |    151/   235 batches
Train set | epoch:   8 |    176/   235 batches
Train set | epoch:   8 |    201/   235 batches
Train set | epoch:   8 |    

Train set | epoch:  12 |    676/   705 batches
Train set | epoch:  12 |    701/   705 batches
Phase:train, epoch loss: 0.0899
Train set | epoch:  12 |      1/   235 batches
Train set | epoch:  12 |     26/   235 batches
Train set | epoch:  12 |     51/   235 batches
Train set | epoch:  12 |     76/   235 batches
Train set | epoch:  12 |    101/   235 batches
Train set | epoch:  12 |    126/   235 batches
Train set | epoch:  12 |    151/   235 batches
Train set | epoch:  12 |    176/   235 batches
Train set | epoch:  12 |    201/   235 batches
Train set | epoch:  12 |    226/   235 batches
Phase:validate, epoch loss: 0.0738 | epoch accuracy: 0.0000
Epoch: 13/99
----------
Train set | epoch:  13 |      1/   705 batches
Train set | epoch:  13 |     26/   705 batches
Train set | epoch:  13 |     51/   705 batches
Train set | epoch:  13 |     76/   705 batches
Train set | epoch:  13 |    101/   705 batches
Train set | epoch:  13 |    126/   705 batches
Train set | epoch:  13 |    151/   705

Train set | epoch:  16 |    151/   235 batches
Train set | epoch:  16 |    176/   235 batches
Train set | epoch:  16 |    201/   235 batches
Train set | epoch:  16 |    226/   235 batches
Phase:validate, epoch loss: 0.0723 | epoch accuracy: 0.0000
Epoch: 17/99
----------
Train set | epoch:  17 |      1/   705 batches
Train set | epoch:  17 |     26/   705 batches
Train set | epoch:  17 |     51/   705 batches
Train set | epoch:  17 |     76/   705 batches
Train set | epoch:  17 |    101/   705 batches
Train set | epoch:  17 |    126/   705 batches
Train set | epoch:  17 |    151/   705 batches
Train set | epoch:  17 |    176/   705 batches
Train set | epoch:  17 |    201/   705 batches
Train set | epoch:  17 |    226/   705 batches
Train set | epoch:  17 |    251/   705 batches
Train set | epoch:  17 |    276/   705 batches
Train set | epoch:  17 |    301/   705 batches
Train set | epoch:  17 |    326/   705 batches
Train set | epoch:  17 |    351/   705 batches
Train set | epoch:  17 

...

Train set | epoch:  59 |     26/   705 batches
Train set | epoch:  59 |     51/   705 batches
Train set | epoch:  59 |     76/   705 batches
Train set | epoch:  59 |    101/   705 batches
Train set | epoch:  59 |    126/   705 batches
Train set | epoch:  59 |    151/   705 batches
Train set | epoch:  59 |    176/   705 batches
Train set | epoch:  59 |    201/   705 batches
Train set | epoch:  59 |    226/   705 batches
Train set | epoch:  59 |    251/   705 batches
Train set | epoch:  59 |    276/   705 batches
Train set | epoch:  59 |    301/   705 batches
Train set | epoch:  59 |    326/   705 batches
Train set | epoch:  59 |    351/   705 batches
Train set | epoch:  59 |    376/   705 batches
Train set | epoch:  59 |    401/   705 batches
Train set | epoch:  59 |    426/   705 batches
Train set | epoch:  59 |    451/   705 batches
Train set | epoch:  59 |    476/   705 batches
Train set | epoch:  59 |    501/   705 batches
Train set | epoch:  59 |    526/   705 batches
Train set | e

Train set | epoch:  63 |    251/   705 batches
Train set | epoch:  63 |    276/   705 batches
Train set | epoch:  63 |    301/   705 batches
Train set | epoch:  63 |    326/   705 batches
Train set | epoch:  63 |    351/   705 batches
Train set | epoch:  63 |    376/   705 batches
Train set | epoch:  63 |    401/   705 batches
Train set | epoch:  63 |    426/   705 batches
Train set | epoch:  63 |    451/   705 batches
Train set | epoch:  63 |    476/   705 batches
Train set | epoch:  63 |    501/   705 batches
Train set | epoch:  63 |    526/   705 batches
Train set | epoch:  63 |    551/   705 batches
Train set | epoch:  63 |    576/   705 batches
Train set | epoch:  63 |    601/   705 batches
Train set | epoch:  63 |    626/   705 batches
Train set | epoch:  63 |    651/   705 batches
Train set | epoch:  63 |    676/   705 batches
Train set | epoch:  63 |    701/   705 batches
Phase:train, epoch loss: 0.0899
Train set | epoch:  63 |      1/   235 batches
Train set | epoch:  63 |    

Train set | epoch:  67 |    476/   705 batches
Train set | epoch:  67 |    501/   705 batches
Train set | epoch:  67 |    526/   705 batches
Train set | epoch:  67 |    551/   705 batches
Train set | epoch:  67 |    576/   705 batches
Train set | epoch:  67 |    601/   705 batches
Train set | epoch:  67 |    626/   705 batches
Train set | epoch:  67 |    651/   705 batches
Train set | epoch:  67 |    676/   705 batches
Train set | epoch:  67 |    701/   705 batches
Phase:train, epoch loss: 0.0899
Train set | epoch:  67 |      1/   235 batches
Train set | epoch:  67 |     26/   235 batches
Train set | epoch:  67 |     51/   235 batches
Train set | epoch:  67 |     76/   235 batches
Train set | epoch:  67 |    101/   235 batches
Train set | epoch:  67 |    126/   235 batches
Train set | epoch:  67 |    151/   235 batches
Train set | epoch:  67 |    176/   235 batches
Train set | epoch:  67 |    201/   235 batches
Train set | epoch:  67 |    226/   235 batches
Phase:validate, epoch loss: 

Train set | epoch:  71 |    701/   705 batches
Phase:train, epoch loss: 0.0899
Train set | epoch:  71 |      1/   235 batches
Train set | epoch:  71 |     26/   235 batches
Train set | epoch:  71 |     51/   235 batches
Train set | epoch:  71 |     76/   235 batches
Train set | epoch:  71 |    101/   235 batches
Train set | epoch:  71 |    126/   235 batches
Train set | epoch:  71 |    151/   235 batches
Train set | epoch:  71 |    176/   235 batches
Train set | epoch:  71 |    201/   235 batches
Train set | epoch:  71 |    226/   235 batches
Phase:validate, epoch loss: 0.0730 | epoch accuracy: 0.0000
Epoch: 72/99
----------
Train set | epoch:  72 |      1/   705 batches
Train set | epoch:  72 |     26/   705 batches
Train set | epoch:  72 |     51/   705 batches
Train set | epoch:  72 |     76/   705 batches
Train set | epoch:  72 |    101/   705 batches
Train set | epoch:  72 |    126/   705 batches
Train set | epoch:  72 |    151/   705 batches
Train set | epoch:  72 |    176/   705

Train set | epoch:  75 |    176/   235 batches
Train set | epoch:  75 |    201/   235 batches
Train set | epoch:  75 |    226/   235 batches
Phase:validate, epoch loss: 0.0954 | epoch accuracy: 0.0000
Epoch: 76/99
----------
Train set | epoch:  76 |      1/   705 batches
Train set | epoch:  76 |     26/   705 batches
Train set | epoch:  76 |     51/   705 batches
Train set | epoch:  76 |     76/   705 batches
Train set | epoch:  76 |    101/   705 batches
Train set | epoch:  76 |    126/   705 batches
Train set | epoch:  76 |    151/   705 batches
Train set | epoch:  76 |    176/   705 batches
Train set | epoch:  76 |    201/   705 batches
Train set | epoch:  76 |    226/   705 batches
Train set | epoch:  76 |    251/   705 batches
Train set | epoch:  76 |    276/   705 batches
Train set | epoch:  76 |    301/   705 batches
Train set | epoch:  76 |    326/   705 batches
Train set | epoch:  76 |    351/   705 batches
Train set | epoch:  76 |    376/   705 batches
Train set | epoch:  76 

In [ ]:
plt.plot(epochLossTrain, label = "Epoch Train Loss")
plt.plot(epochLossValidate, label = "Epoch Validate Loss")
plt.legend(loc = "upper right")

In [ ]:
# plt.plot(epochLossTrain, label = "Epoch Train Loss")
plt.plot(epochAccValidate, label = "Epoch Accuracy")
plt.legend(loc = "upper right")